In [ ]:
import pandas as pd 
from pathlib2 import Path
import plotly.graph_objects as go

In [ ]:
DEFAULT_PI_PATH = Path('/home/pi/get_temp_C.out')
FALLBACK_PATH = Path('/Users/eandreas/projects/dev/ds18b20/webapp2/data/get_temp_C.out')

In [ ]:
def load_data():
    path = DEFAULT_PI_PATH
    if not path.is_file():
        path = FALLBACK_PATH
    names = ['dev_sn', 'date', 'time', 'temp_raw', 'temp_C']
    df = pd.read_csv(path, sep=' ', header=None, names = names, parse_dates=[['date', 'time']])
    
    # keep only rows witout nan entries
    df = df[~df.isna().any(axis=1)]
    
    dfs = {}
    
    for sn in df[df.temp_raw.notna()].dev_sn.unique():
        # create a copy for each device / serial number
        dfd = df[df.dev_sn == sn].copy()
        # reset index due to skipped rows (different serial number)
        dfd = dfd.reset_index(drop = True)
        # keep only dates, hours, and minutes from date_time column
        dfd.date_time = dfd.date_time.dt.strftime('%Y-%m-%d %H:%M')
        dfd.date_time = pd.to_datetime(dfd['date_time'], format='%Y%m%d %H:%M')
        #dfd = dfd[~dfd.isna().any(axis=1)]
        # remov duplicate rows for the same time stamp
        dfd = dfd[~dfd.date_time.duplicated(keep='first')]
        # fill gaps in case of missing measured data points
        idx = pd.date_range(
            start = dfd.date_time.iloc[0].strftime('%Y-%m-%d %H:%M'),
            end = dfd.date_time.iloc[-1].strftime('%Y-%m-%d %H:%M'),
            freq = 'T')
        dfd = dfd.set_index('date_time').reindex(idx).rename_axis('date_time').reset_index()
        # store within dictionary
        dfs.update({sn: dfd})
        
    return dfs

In [ ]:
def serve_figure(df):    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=list(df.date_time),
        y=list(df.temp_C),
        name='t_corr',
        line=dict(color='skyblue', width=2, dash='solid'),
        #fill='tozeroy',
    ))
    fig.add_trace(go.Scatter(
        x=list(df.date_time),
        y=list(df.temp_raw),
        name='t_raw',
        line=dict(color='darkgray', width=1, dash='dot')
    ))
    fig.update_layout(
        xaxis_title="Datum",
        yaxis_title="Temperatur / °C",
        template='none',
        #autosize=False,
        #width=1000,
        #height=500,
        margin=dict(
            l=60,
            r=0,
            b=0,
            t=0,
            pad=4
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.9
        )
    )
    return fig

In [ ]:
df = load_data()
#serve_figure(df)

In [ ]:
for sn in df.keys():
    print(df[sn].tail())

                date_time           dev_sn  temp_raw  temp_C
23054 2021-02-15 22:02:00  28-03219779d339    17.375  18.829
23055 2021-02-15 22:03:00  28-03219779d339    17.375  18.829
23056 2021-02-15 22:04:00  28-03219779d339    17.312  18.767
23057 2021-02-15 22:05:00  28-03219779d339    17.312  18.767
23058 2021-02-15 22:06:00  28-03219779d339    17.250  18.705
               date_time           dev_sn  temp_raw  temp_C
4299 2021-02-15 22:02:00  28-032197791b3c    16.875  16.875
4300 2021-02-15 22:03:00  28-032197791b3c    16.875  16.875
4301 2021-02-15 22:04:00  28-032197791b3c    16.875  16.875
4302 2021-02-15 22:05:00  28-032197791b3c    16.875  16.875
4303 2021-02-15 22:06:00  28-032197791b3c    16.812  16.812


In [ ]:
list(df.keys())[0]

'28-03219779d339'